<div style="text-align: center; background-color: lightblue; padding: 15px;">
    <p style="text-align: center;">بسم الله، والحمد لله، والصلاة والسلام على رسول الله وعلى آله وصحبه</p>
</div>

<div style="border: 2px solid black; padding: 10px;">
    <h2 style="text-align: center;">CUI</h2>
    <p><strong>PREPARE PAR L' ELEVE INGENIEUR:</strong></p>
    <ul>
        <li>Afaf ALOUI</li>
        <li>Fadoua HAIDA</li>
        <li>MOHAMED ELBADRI</li>
    </ul>
</div>

<div class="note" style="background-color: lightyellow; color: black; padding: 10px;">
  <h4> A- LOAD REQUIRED LIBRARIES </h4>
</div>

In [1]:
import pandas as pd
import numpy as np
import cartopy.crs as ccrs 
import cartopy.feature as cfeat
import matplotlib.pyplot as plt
import xarray as xr
import os
import metpy.constants as mpconst
from metpy.units import units 
import seaborn as sns
import xskillscore as xs
import matplotlib.dates as mdates
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
# widgets.IntSlider()

<div class="note" style="background-color: lightyellow; color: black; padding: 10px;">
  <h4> B- PREPARE INPUTS </h4>
</div>

<div class="note" style="background-color: teal; color: white; padding: 10px; border-radius: 5px;">
  <h4> B-1 CITIES </h4>
</div>

In [2]:
CITY = {
    'Dakhla': 15.5,
    'Laayoune': 19.5,
    'Tarfaya': 21.5,
    'Agadir': 23.5,
    'Essaouira': 25.5,
    'El Jadida': 27.5,
    'Casablanca': 29.5,
    'Mohammedia': 31.5,
    'Rabat': 32.5,
    'Skhirat': 33.5,
    'Tangier': 35.5,
    'Asilah': 37.5,
    'Larache': 39.5,
    'Ksar es seghir': 41.5
}

<div class="note" style="background-color: teal; color: white; padding: 10px; border-radius: 5px;">
  <h4> B-2 NAO </h4>
</div>

In [3]:
DATAIN = "/home/mohamed/EHTPIII/CLIMATOLOGIE/DATA/INOUT/"
def load_nao_data(file_path):
    """
    Charge et prépare les données NAO à partir d'un fichier texte.
    
    Args:
        file_path (str): Chemin vers le fichier NAO
        
    Returns:
        pd.DataFrame: DataFrame contenant les données NAO mensuelles indexées par date
    """
    nao_data = pd.read_csv(file_path, delim_whitespace=True)
    nao_monthly = pd.DataFrame()
    
    for year in nao_data['YEAR'].unique():
        year_data = nao_data[nao_data['YEAR'] == year]
        for month in ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                     'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']:
            date = pd.to_datetime(f"{year}-{month}-01").strftime("%Y-%m")  
            nao_monthly.loc[date, 'NAO'] = year_data[month].values[0]
    
    # Réinitialiser l'index pour ajouter la colonne 'Date'
    nao_monthly.reset_index(inplace=True)
    nao_monthly.rename(columns={'index': 'Date'}, inplace=True)  # Renommer la colonne d'index

    # Convertir la colonne 'Date' en datetime
    nao_monthly['Date'] = pd.to_datetime(nao_monthly['Date'])


    return nao_monthly

# Utiliser la fonction pour charger les données NAO
nao_monthly = load_nao_data(DATAIN+"NAO.txt")

# Sauvegarder dans un fichier CSV avec le séparateur tab
nao_monthly.to_csv(DATAIN+"NAO_2.csv", sep=',', index=False)

/tmp/ipykernel_4073/2101350512.py:12: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  nao_data = pd.read_csv(file_path, delim_whitespace=True)


In [4]:
NAO=pd.read_csv(DATAIN+'NAO_2.csv')
NAO['Date'] = pd.to_datetime(NAO['Date'])
NAO.Date=NAO.Date.dt.to_period("M")
NAO


,Date,NAO
0,1950-01,0.92
1,1950-02,0.40
2,1950-03,-0.36
3,1950-04,0.73
4,1950-05,-0.59
...,...,...
895,2024-08,0.63
896,2024-09,NaN
897,2024-10,NaN
898,2024-11,NaN


<div class="note" style="background-color: teal; color: white; padding: 10px; border-radius: 5px;">
  <h4> B-3 CUI </h4>
</div>

In [40]:
UI=pd.read_csv("/home/mohamed/EHTPIII/CLIMATOLOGIE/DATA/UI.csv")
UI['longitude'] = pd.to_numeric(UI['longitude'], errors='coerce')
UI=UI.iloc[1:]
ektrx=pd.to_numeric(UI.ektrx,errors='coerce')
ektry=pd.to_numeric(UI.ektry,errors='coerce')
coast_angle=135

def upwell(ektrx, ektry, coast_angle):
  import numpy as np
  pi = 3.1415927
  degtorad = pi/180.
  alpha = (360 - coast_angle) * degtorad
  s1 = np.cos(alpha)
  t1 = np.sin(alpha)
  s2 = -1 * t1
  t2 = s1
  perp = (s1 * ektrx) + (t1 * ektry)
  para = (s2 * ektrx) + (t2 * ektry)
  return(perp/10)


CUI=UI.copy()
CUI["CUI"]=upwell(ektrx, ektry, coast_angle)
CUI.time=pd.to_datetime(CUI.time)
CUI["latitude"]=pd.to_numeric(CUI["latitude"],errors='coerce')
CUI=CUI.groupby(['time','latitude'])['CUI'].mean().reset_index()
CUI=CUI[CUI['latitude'].isin(CITY.values())]
latitude_to_city={v: k for k, v in CITY.items()}
CUI["city"]=CUI['latitude'].map(latitude_to_city)
CUI["time"]=pd.to_datetime(CUI["time"]).dt.to_period("M")
CUI["time"]=CUI["time"].dt.to_timestamp()
CUI['city'] = CUI['city'].astype('string')
CUI

/tmp/ipykernel_4073/2282613933.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  UI=pd.read_csv("/home/mohamed/EHTPIII/CLIMATOLOGIE/DATA/UI.csv")
/tmp/ipykernel_4073/2282613933.py:30: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  CUI["time"]=pd.to_datetime(CUI["time"]).dt.to_period("M")


,time,latitude,CUI,city
0,1967-01-01,15.5,-332.506976,Dakhla
4,1967-01-01,19.5,-126.363259,Laayoune
6,1967-01-01,21.5,-88.743291,Tarfaya
8,1967-01-01,23.5,-85.539639,Agadir
10,1967-01-01,25.5,-79.480394,Essaouira
...,...,...,...,...
19422,2024-10-01,33.5,15.153246,Skhirat
19424,2024-10-01,35.5,23.008839,Tangier
19426,2024-10-01,37.5,24.565583,Asilah
19428,2024-10-01,39.5,24.788460,Larache


In [26]:
NAO_filtered = NAO[(NAO['Date'] >= CUI['time'].min()) & (NAO['Date'] <= CUI['time'].max())]
NAO_filtered

,Date,NAO
204,1967-01,-0.89
205,1967-02,0.19
206,1967-03,1.51
207,1967-04,0.18
208,1967-05,-0.99
...,...,...
893,2024-06,-0.09
894,2024-07,1.46
895,2024-08,0.63
896,2024-09,NaN


<div class="note" style="background-color: lightyellow; color: black; padding: 10px;">
  <h4> C- PLOTS</h4>
</div>

In [93]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_cui(city):

    fig, ax = plt.subplots(figsize=(25, 10))
    DF=CUI.query("city==@city")

    ax=sns.lineplot(data=DF, x="time", y="CUI", ax=ax, label="CUI", color="tab:blue")
    ax.set_ylabel("CUI", fontsize=14)
    ax.set_ylim(DF["CUI"].min() * 1.2, DF["CUI"].max() * 1.2) 
    ax.tick_params(axis='y', labelcolor="tab:blue")
    
    ax2 = ax.twinx()
    ax2=sns.lineplot(data=NAO_filtered, x="Date", y="NAO", ax=ax2, label="NAO", color="tab:red")
    ax2.set_ylabel("NAO", fontsize=14)
    ax2.set_ylim(NAO_filtered["NAO"].min() * 1.2, NAO_filtered["NAO"].max() * 1.2) 
    ax2.tick_params(axis='y', labelcolor="tab:red")
    
    plt.title("CUI and NAO Over Time", fontsize=18)
    ax.legend(loc="upper left", fontsize=12)
    ax2.legend(loc="upper right", fontsize=12)
    plt.show()
city_dropdown=widgets.Dropdown(options=CITY.keys(),value="Dakhla")
widgets.interactive(plot_cui,city=city_dropdown)

interactive(children=(Dropdown(description='city', options=('Dakhla', 'Laayoune', 'Tarfaya', 'Agadir', 'Essaou…

In [94]:
# Extract the Upwelling Index for Tangier using the coordinates for longitude and latitude
def UIC_PLOT(city):
    fig,ax=plt.subplots(figsize=(14, 7))
    DF=CUI.copy()
    cui=DF.query("city==@city").loc[:,"CUI"]
    t=DF.query("city==@city").loc[:,"time"]
    colors = ['lightblue' if val >= 0 else 'orange' for val in cui]
    nao=NAO_filtered.NAO

    ax.bar(t, cui, color=colors, width=5.0)
    ax.axhline(0, color='black', linewidth=2)
    ax.set_title(f'Upwelling Index for {city} vs NAO' , fontsize=16)
    ax.set_xlabel('Date', fontsize=14)
    ax.set_ylabel('Upwelling Index', fontsize=14)
    ax.set_ylim(cui.min() * 1.2 , cui.max() * 1.2)
    ax.grid(visible=True, linestyle='--', alpha=0.2)
    
city_dropdown = widgets.Dropdown(options=CITY.keys(),description='City')
widgets.interactive(UIC_PLOT, city=city_dropdown)

interactive(children=(Dropdown(description='City', options=('Dakhla', 'Laayoune', 'Tarfaya', 'Agadir', 'Essaou…